# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load data from weather

vacation_df = pd.read_csv("clean_city_data.csv")


In [4]:
vacation_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,barrow,71.29,-156.79,24.80,68,90,17.22,US,1602430919
1,1,hermanus,-34.42,19.23,64.99,53,99,10.00,ZA,1602430959
2,2,tiksi,71.69,128.87,12.00,94,46,9.28,RU,1602430959
3,3,atuona,-9.80,-139.03,78.26,72,24,24.49,PF,1602430959
4,4,katsuura,35.13,140.30,68.00,88,75,13.87,JP,1602430959


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps

fig = gmaps.configure(api_key=g_key)

# Create variables for coordinates and humidity

locations = vacation_df[['Lat', 'Lng']]
weights = vacation_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 70, Wind speed less than 10 mph, Zero cloudiness.

weather_conditions_df = vacation_df.loc[(vacation_df["Max Temp"] >70) & (vacation_df["Max Temp"] <80) & (vacation_df["Cloudiness"] ==0) & (vacation_df["Wind Speed"] <10)]
weather_conditions_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,46,caraz,-9.05,-77.81,74.53,20,0,3.13,PE,1602430962
169,173,pyapon,16.28,95.68,78.80,85,0,8.16,MM,1602430971
294,300,ramhormoz,31.28,49.60,75.06,19,0,2.62,IR,1602430981
326,333,tezu,27.92,96.17,76.68,71,0,3.24,IN,1602430983
343,351,alyangula,-13.85,136.42,77.00,83,0,2.24,AU,1602430985
344,352,gallipoli,40.41,26.67,75.20,47,0,4.70,TR,1602430985
397,409,zurrieq,35.83,14.47,75.20,60,0,9.17,MT,1602430989
419,431,bandarbeyla,9.49,50.81,79.66,74,0,6.02,SO,1602430990
439,451,tucuman,-26.82,-65.22,75.20,23,0,9.17,AR,1602430992
519,536,marsa matruh,31.35,27.25,74.82,63,0,9.55,EG,1602430932


In [11]:
# Create DataFrame called hotel_df to store hotel names with the best conditions

hotel_df = weather_conditions_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
44,caraz,PE,-9.05,-77.81,
169,pyapon,MM,16.28,95.68,
294,ramhormoz,IR,31.28,49.60,
326,tezu,IN,27.92,96.17,
343,alyangula,AU,-13.85,136.42,
344,gallipoli,TR,40.41,26.67,
397,zurrieq,MT,35.83,14.47,
419,bandarbeyla,SO,9.49,50.81,
439,tucuman,AR,-26.82,-65.22,
519,marsa matruh,EG,31.35,27.25,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.